<a href="https://colab.research.google.com/github/tousifo/ml_notebooks/blob/main/Derma_of_MedMNIST_QNN_AllInOne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ============================================================================
# SNIPPET 1: Setup =====================

!pip install -q medmnist pennylane pennylane-lightning torch torchvision scikit-learn pandas tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.3/934.3 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 7.7 MB/s eta 0:00:00


In [5]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
import torchvision.transforms as T
import torchvision.models as models
from torchvision.transforms import InterpolationMode
from torch.optim.swa_utils import AveragedModel, update_bn, SWALR
import pennylane as qml
import medmnist
from medmnist import INFO
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from tqdm import tqdm
from PIL import ImageFilter
import warnings
import scipy.integrate

warnings.filterwarnings("ignore")

# ==========================================
# 1. DERMAMNIST CONFIGURATION (High-Performance)
# ==========================================
CONFIG = {
    "dataset": "dermamnist",
    "img_size": 224,         # Upscale to 224 for ResNet18 backbone compatibility
    "D": 512,                # Embedding Dimension (ResNet output)
    "q_wires": 6,            # 6 Qubits
    "q_layers": 3,           # 3 Layer Ansatz
    "head_hidden": 128,
    "dropout": 0.3,
    "n_classes": 7,

    # Training Hyperparams
    "batch_size": 32,        # Smaller batch for better generalization
    "epochs": 50,            # Needs time to fine-tune
    "lr": 1e-4,              # Lower LR for fine-tuning
    "wd": 1e-3,              # Higher decay to prevent overfitting
    "label_smoothing": 0.1,
    "seed": 42,

    # Stabilization
    "use_swa": True,
    "swa_start": 40,         # Late SWA
    "device": "cuda" if torch.cuda.is_available() else "cpu"
}

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

set_seed(CONFIG['seed'])
print(f"🔧 Config: DermaMNIST (224x224), ResNet Backbone, SWA Enabled")

🔧 Config: DermaMNIST (7 Classes), 64x64, SWA Starts Epoch 30


In [6]:
# ==========================================
# 2. DATA PIPELINE (Transfer Learning Ready)
# ==========================================
def get_transforms(img_size, is_train):
    # ImageNet stats for Pre-trained Backbone
    mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]

    if is_train:
        return T.Compose([
            T.Resize((img_size, img_size), interpolation=InterpolationMode.BILINEAR),
            T.RandomHorizontalFlip(),
            T.RandomVerticalFlip(),
            T.RandomRotation(20),
            T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
            T.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
            T.ToTensor(),
            T.Normalize(mean, std)
        ])
    return T.Compose([
        T.Resize((img_size, img_size), interpolation=InterpolationMode.BILINEAR),
        T.ToTensor(),
        T.Normalize(mean, std)
    ])

def get_loaders(config):
    info = INFO[config['dataset']]
    DataClass = getattr(medmnist, info['python_class'])

    train_ds = DataClass(split='train', transform=get_transforms(config['img_size'], True), download=True)
    val_ds = DataClass(split='val', transform=get_transforms(config['img_size'], False), download=True)
    test_ds = DataClass(split='test', transform=get_transforms(config['img_size'], False), download=True)

    # Critical: Handle Class Imbalance
    targets = train_ds.labels.squeeze()
    class_counts = np.bincount(targets)
    class_weights = 1. / class_counts
    samples_weights = class_weights[targets]
    sampler = WeightedRandomSampler(samples_weights, len(samples_weights))

    train_loader = DataLoader(train_ds, batch_size=config['batch_size'], sampler=sampler, num_workers=2)
    val_loader = DataLoader(val_ds, batch_size=config['batch_size'], shuffle=False, num_workers=2)
    test_loader = DataLoader(test_ds, batch_size=config['batch_size'], shuffle=False, num_workers=2)

    return train_loader, val_loader, test_loader

print("📦 Data Pipeline Ready (ResNet Norm + Weighted Sampler)")

📦 Data Pipeline Ready: Weighted Sampler + Color Invariance


In [7]:
# ==========================================
# 3. UNIFIED ARCHITECTURE MODULES
# ==========================================

class PatchEmbedding(nn.Module):
    def __init__(self, in_ch=3, D=128, patch=8):
        super().__init__()
        self.proj = nn.Conv2d(in_ch, D, kernel_size=patch, stride=patch)
        self.norm = nn.LayerNorm(D)

    def forward(self, x):
        # [B, C, H, W] -> [B, D, H/P, W/P] -> [B, N, D]
        x = self.proj(x).flatten(2).transpose(1, 2)
        x = self.norm(x)
        return x

class SimpleRNN(nn.Module):
    """BiGRU over patch sequence"""
    def __init__(self, input_dim=128, hidden_dim=128):
        super().__init__()
        # Bidirectional outputs 2*hidden, so we use hidden//2 to keep output size D
        self.rnn = nn.GRU(input_dim, hidden_dim // 2, num_layers=1,
                          batch_first=True, bidirectional=True)

    def forward(self, x):
        # x: [B, N, D] -> [B, N, D]
        out, _ = self.rnn(x)
        # Global Average Pooling over sequence
        return out.mean(dim=1)

class FastQuantumLayer(nn.Module):
    """Proj6 -> Clamp -> RY-CNOT-RZ Ansatz"""
    def __init__(self, in_dim=128, n_qubits=6, n_layers=3):
        super().__init__()
        self.n_qubits = n_qubits
        self.proj6 = nn.Linear(in_dim, n_qubits)
        self.dev = qml.device('default.qubit', wires=n_qubits)
        self.q_params = nn.Parameter(torch.randn(n_layers, n_qubits, 2) * 0.1)
        self.n_layers = n_layers
        self.qnode = qml.QNode(self._circuit, self.dev, interface='torch')

    def _circuit(self, inputs, weights):
        for i in range(self.n_qubits): qml.RY(inputs[i], wires=i)
        for l in range(self.n_layers):
            for i in range(self.n_qubits): qml.RY(weights[l, i, 0], wires=i)
            for i in range(self.n_qubits): qml.CNOT(wires=[i, (i + 1) % self.n_qubits])
            for i in range(self.n_qubits): qml.RZ(weights[l, i, 1], wires=i)
        return [qml.expval(qml.PauliZ(i)) for i in range(self.n_qubits)]

    def forward(self, x):
        a6 = self.proj6(x)
        a6 = torch.clamp(a6, min=-np.pi/2, max=np.pi/2)
        bsz = x.shape[0]
        q_out = [torch.stack(self.qnode(a6[i], self.q_params)) for i in range(bsz)]
        q_vals = torch.stack(q_out).to(x.device).float()
        return q_vals, a6

class FastHybridQNN(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.patch_embed = PatchEmbedding(in_ch=3, D=config['D'], patch=config['patch'])
        self.rnn = SimpleRNN(input_dim=config['D'], hidden_dim=config['D'])
        self.q_layer = FastQuantumLayer(in_dim=config['D'], n_qubits=config['q_wires'], n_layers=config['q_layers'])

        self.head = nn.Sequential(
            nn.Linear(config['D'] + config['q_wires'], config['head_hidden']),
            nn.ReLU(),
            nn.Dropout(config['dropout']),
            nn.Linear(config['head_hidden'], config['n_classes'])
        )
        self.cache = {}

    def forward(self, x):
        x_patch = self.patch_embed(x)
        x_classical = self.rnn(x_patch)
        x_quant, a6 = self.q_layer(x_classical)

        if not self.training:
            self.cache['a6'] = a6

        combined = torch.cat([x_classical, x_quant], dim=1)
        return self.head(combined)

print("🧠 Unified Model Built (BiGRU + 6-Qubit Ring Ansatz)")

🧠 Unified Model Built (BiGRU + 6-Qubit Ring Ansatz)


In [8]:
# ==========================================
# 4. TRAINING ENGINE
# ==========================================
class LabelSmoothingLoss(nn.Module):
    def __init__(self, smoothing=0.05):
        super().__init__()
        self.smoothing = smoothing
    def forward(self, pred, target):
        log_prob = F.log_softmax(pred, dim=-1)
        model_prob = torch.ones_like(log_prob) * (self.smoothing / (pred.size(-1) - 1))
        model_prob.scatter_(1, target.unsqueeze(1), 1.0 - self.smoothing)
        return (-model_prob * log_prob).sum(dim=-1).mean()

def train_engine(model, train_loader, val_loader, config):
    optimizer = optim.AdamW(model.parameters(), lr=config['lr'], weight_decay=config['wd'])
    criterion = LabelSmoothingLoss(config['label_smoothing'])
    swa_model = AveragedModel(model)
    swa_scheduler = SWALR(optimizer, swa_lr=0.0005)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config['epochs'])

    best_acc = 0.0
    print("🚀 Training Started...")

    for epoch in range(config['epochs']):
        model.train()
        for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=False):
            x, y = x.to(config['device']), y.squeeze().long().to(config['device'])
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()

        if config['use_swa'] and epoch >= config['swa_start']:
            swa_model.update_parameters(model)
            swa_scheduler.step()
        else:
            scheduler.step()

        model.eval()
        preds, targets = [], []
        with torch.no_grad():
            for x, y in val_loader:
                x = x.to(config['device'])
                out = model(x)
                preds.append(out.argmax(1).cpu())
                targets.append(y.squeeze())

        acc = accuracy_score(torch.cat(targets), torch.cat(preds))
        if acc > best_acc: best_acc = acc
        print(f"Epoch {epoch+1} | Val Acc: {acc:.4f}")

    print("🔄 SWA Finisher: Updating Batch Norm...")
    update_bn(train_loader, swa_model, device=config['device'])
    return swa_model

# --- EXECUTION ---
train_loader, val_loader, test_loader = get_loaders(CONFIG)
model = FastHybridQNN(CONFIG).to(CONFIG['device'])
final_model = train_engine(model, train_loader, val_loader, CONFIG)

# Final Accuracy
final_model.eval()
preds, targets = [], []
with torch.no_grad():
    for x, y in test_loader:
        x = x.to(CONFIG['device'])
        out = final_model(x)
        preds.append(out.argmax(1).cpu())
        targets.append(y.squeeze())
print(f"\n🏆 Final Test Accuracy: {accuracy_score(torch.cat(targets), torch.cat(preds)):.4f}")

🚀 Training Started...


Epoch 1 | Val Acc: 0.2453


Epoch 2 | Val Acc: 0.4566


Epoch 3 | Val Acc: 0.3579


Epoch 4 | Val Acc: 0.4706


...

Epoch 41 | Val Acc: 0.6361


Epoch 42 | Val Acc: 0.6341


Epoch 43 | Val Acc: 0.6391


Epoch 44 | Val Acc: 0.6321


Epoch 45 | Val Acc: 0.6451
🔄 SWA Finisher: Updating Batch Norm...

🏆 Final Test Accuracy: 0.6409


In [9]:
# ==========================================
# 5. XAI ENGINE (Tri-Layer Stack)
# ==========================================
class XAIEngine:
    def __init__(self, model, device):
        self.device = device
        # Unwrap SWA model
        self.model = model.module if hasattr(model, 'module') else model
        self.gradients, self.activations = None, None
        self.target_layer = self.model.patch_embed.proj
        self.target_layer.register_forward_hook(self.save_act)
        self.target_layer.register_full_backward_hook(self.save_grad)

    def save_act(self, module, input, output): self.activations = output
    def save_grad(self, module, grad_in, grad_out): self.gradients = grad_out[0]

    def normalize(self, x):
        return (x - x.min()) / (x.max() - x.min() + 1e-8)

    def grad_cam(self, x, class_idx):
        self.model.eval(); self.model.zero_grad()
        out = self.model(x)
        out[0, class_idx].backward(retain_graph=True)
        grad = self.gradients.mean(dim=(2, 3), keepdim=True)
        cam = (grad * self.activations).sum(dim=1, keepdim=True)
        return self.normalize(F.interpolate(F.relu(cam), size=x.shape[2:], mode='bilinear'))

    def integrated_gradients(self, x, class_idx, steps=20):
        baseline = torch.zeros_like(x)
        scaled = torch.cat([baseline + (float(i)/steps)*(x-baseline) for i in range(steps+1)], 0)
        scaled.requires_grad_(True)
        preds = self.model(scaled)
        score = preds[:, class_idx].sum()
        self.model.zero_grad(); score.backward()
        ig = (x - baseline) * torch.mean(scaled.grad, dim=0)
        return self.normalize(ig.sum(dim=1, keepdim=True).abs())

    def q_saliency(self, x, class_idx):
        self.model.eval(); x.requires_grad_(True)
        logits = self.model(x)
        if 'a6' not in self.model.cache: _ = self.model(x)
        a6 = self.model.cache['a6']; a6.retain_grad()
        logits[0, class_idx].backward(retain_graph=True)
        da6 = a6.grad.detach().abs()
        W = self.model.q_layer.proj6.weight.detach().abs()
        S_feat = torch.matmul(da6, W)
        patch_acts = self.activations.detach().abs()
        S_patch = torch.einsum('bd,bdhw->bhw', S_feat, patch_acts).unsqueeze(1)
        return self.normalize(F.interpolate(S_patch, size=x.shape[2:], mode='bilinear'))

    def mc_dropout_entropy(self, x, T=10):
        self.model.train()
        outputs = torch.stack([torch.softmax(self.model(x), dim=1) for _ in range(T)])
        mean_p = outputs.mean(dim=0)
        return -torch.sum(mean_p * torch.log(mean_p + 1e-8), dim=1).item()

    def deletion_auc(self, x, target, saliency_map):
        B, C, H, W = x.shape
        flat_sal = saliency_map.view(B, -1)
        sorted_idx = torch.argsort(flat_sal, descending=True, dim=1)
        probs = []
        steps = np.linspace(0, 0.5, 10)
        for frac in steps:
            k = int(frac * H * W)
            mask = torch.ones_like(flat_sal)
            if k > 0: mask.scatter_(1, sorted_idx[:, :k], 0.0)
            with torch.no_grad():
                out = self.model(x * mask.view(B, 1, H, W))
                probs.append(F.softmax(out, dim=1)[0, target].item())
        return scipy.integrate.trapezoid(probs, dx=1.0/len(steps))

# Execute XAI
print("\n🔍 Generating XAI Panel...")
xai = XAIEngine(final_model, CONFIG['device'])
x, y = next(iter(test_loader))
x = x[:1].to(CONFIG['device']); target = y[0].item()

q_map = xai.q_saliency(x, target)
titles = ["Input", "Grad-CAM++", "Integrated Grads", "Q-Saliency"]
maps = [x, xai.grad_cam(x, target), xai.integrated_gradients(x, target), q_map]

plt.figure(figsize=(12, 3))
for i in range(4):
    plt.subplot(1, 4, i+1)
    img_disp = x.squeeze().detach().cpu().permute(1, 2, 0) * 0.5 + 0.5
    if i == 0: plt.imshow(img_disp)
    else:
        plt.imshow(img_disp)
        plt.imshow(maps[i].squeeze().detach().cpu(), cmap='jet', alpha=0.5)
    plt.title(titles[i]); plt.axis('off')

print(f"   Entropy: {xai.mc_dropout_entropy(x):.4f}")
print(f"   Deletion AUC: {xai.deletion_auc(x, target, q_map):.4f}")
plt.show()


🔍 Generating XAI Panel...


TypeError: mean() received an invalid combination of arguments - got (NoneType, dim=int), but expected one of:
 * (Tensor input, *, torch.dtype dtype = None, Tensor out = None)
 * (Tensor input, tuple of ints dim, bool keepdim = False, *, torch.dtype dtype = None, Tensor out = None)
 * (Tensor input, tuple of names dim, bool keepdim = False, *, torch.dtype dtype = None, Tensor out = None)
